In [ ]:
!pip install --upgrade pip
!pip install torch transformers gradio requests

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [ ]:
import requests
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Load the MusicGen model
pipe = pipeline("text-to-audio", model="facebook/musicgen-small")

# Define Deezer API search endpoint
DEEZER_API_URL = "https://api.deezer.com/search"

# Function to search for songs
def search_song(query):
    params = {
        'q': query
    }
    response = requests.get(DEEZER_API_URL, params=params)
    if response.status_code == 200:
        return response.json().get('data', [])
    else:
        return []

# Function to generate music
def generate_music(prompt):
    inputs = pipe(prompt)
    return inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
import gradio as gr

def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Project Sonifyr")

        # Search bar for song or artist
        query = gr.Textbox(label="Search for a song or artist:")

        # Button to search for songs
        search_btn = gr.Button("Search Songs")

        # Output area for song results
        song_output = gr.Markdown("")

        # Button to generate music
        generate_btn = gr.Button("Generate Music")

        # Output area for generated music
        generated_output = gr.Markdown("")

        # Functionality for searching songs
        def on_search(query):
            songs = search_song(query)
            if songs:
                results = f"Found {len(songs)} results for '{query}':\n"
                for song in songs:
                    song_title = song['title']
                    artist_name = song['artist']['name']
                    preview_url = song['preview']
                    results += f"- {song_title} by {artist_name}\n"
                    results += f"[Listen Preview]({preview_url})\n"
                return results
            else:
                return "No songs found. Try a different search term."

        # Functionality for generating music
        def on_generate_music(prompt):
            generated_music = generate_music(prompt)
            return f"Generated Music:\n{generated_music}"

        # Connect buttons to functions
        search_btn.click(on_search, inputs=query, outputs=song_output)
        generate_btn.click(on_generate_music, inputs=query, outputs=generated_output)

    return demo

# Create the interface
iface = create_interface()

In [ ]:
# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9926be9948f3dec425.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
